<a href="https://colab.research.google.com/github/dimoynwa/Computer-vision-tasks/blob/main/tennis_court_key_points_pytorch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the dataset.

The dataset consists of 8841 images, which were separeted to train set (75%) and validation set (25%). Each image has 14 annotated points. The resolution of images is 1280×720. This dataset contains all court types (hard, clay, grass).

In [ ]:
!wget "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=7d0a40b1-c2e2-4e46-ab88-088674963ef0&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040" -c -O 'tennis_court_det_dataset.zip'
!unzip "tennis_court_det_dataset.zip"

--2024-04-17 19:57:05--  https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=7d0a40b1-c2e2-4e46-ab88-088674963ef0&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.31.132, 2a00:1450:4013:c1a::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.31.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2004 (2.0K) [text/html]
Saving to: ‘tennis_court_det_dataset.zip’

tennis_court_det_da 100%[===================>]   1.96K  --.-KB/s    in 0s      

2024-04-17 19:57:05 (29.9 MB/s) - ‘tennis_court_det_dataset.zip’ saved [2004/2004]

Archive:  tennis_court_det_dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/TFOD'
!unzip '/content/drive/MyDrive/Colab Notebooks/TFOD/tennis_court_det_dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: data/images/JNKp7sCdQlY_2200.png  
  inflating: data/images/JNKp7sCdQlY_2250.png  
  inflating: data/images/JNKp7sCdQlY_250.png  
  inflating: data/images/JNKp7sCdQlY_300.png  
  inflating: data/images/JNKp7sCdQlY_350.png  
  inflating: data/images/JNKp7sCdQlY_400.png  
  inflating: data/images/JNKp7sCdQlY_450.png  
  inflating: data/images/JNKp7sCdQlY_50.png  
  inflating: data/images/JNKp7sCdQlY_500.png  
  inflating: data/images/JNKp7sCdQlY_550.png  
  inflating: data/images/JNKp7sCdQlY_600.png  
  inflating: data/images/JNKp7sCdQlY_650.png  
  inflating: data/images/JNKp7sCdQlY_700.png  
  inflating: data/images/JNKp7sCdQlY_750.png  
  inflating: data/images/juXbdW7z0WA_100.png  
  inflating: data/images/juXbdW7z0WA_1050.png  
  inflating: data/images/juXbdW7z0WA_1100.png  
  inflating: data/images/juXbdW7z0WA_200.png  
  inflating: data/images/juXbdW7z0WA_350.png  
  inflating: data/images/juXbdW7z0WA_400.png  
  infl

In [ ]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 38.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

## Creating Pytorch model

We will create a Pytorch model to detect tennis court key points.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
# Import the model
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [ ]:
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_str)
print(f'Device: {device}')

Device: cuda


## Create Torch dataset

We will use images and labels, we have downloaded into ```data``` folder and
create a Torch dataset.

Our dataser contains files(images) which are PNG images and ```data_val.json```. In this data file  we have a JSON array with objects.
Every object contains ``id`` which is image ID (file name), ``kps`` which is an array of arrays of 2 elements (**Key points of the court**). There are 14 key points for every image and the array is the coordinates of the key point.

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file, img_size=224) -> None:
        self.img_dir = img_dir
        self.data_file = data_file
        self.img_size = img_size
        with open(data_file, 'r') as df:
            self.data = json.load(df)

        # We need  also a transformation function
        # Purpose of the transofrmation function is to standartize the image size and normalize the input
        # 1. convert to PIL image
        # 2. resize to (224, 224)
        # 3. convert PIL image to torch tensor
        # 4. normalize the tensor, so that values are easier to train
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[.485, .456, .406], std=[.229, .224, .225])
        ])

    def __len__(self):
        return len(self.data)

    # Get the item at given index
    # In the training we are going to get items 1 by 1 or batch_size by batch_size
    def __getitem__(self, index):
        item = self.data[index]
        item_id = item['id']
        img = cv2.imread(f'{self.img_dir}/{item_id}.png')
        height, width = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)

        # arr.flatten() Convert nD to 1D array
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        # Now after the resize of the image, we also need to remap the keypoints locations
        # Let's say our original width is 240 and we resize it to 224. How to find the new key point width

        # Adjust every second element - x coordinate
        kps[::2] *= self.img_size / width
        # Adjust every second element starting from index 1 - y coordinate
        kps[1::2] *= self.img_size / height

        return img, kps


In [ ]:
kpss = np.array([[1,2], [1,2]]).flatten().astype(np.float32)

h, w = 2, 2

kpss[::2] *= 120 / w
kpss[1::2] *= 120 / h

kpss

array([ 60., 120.,  60., 120.], dtype=float32)

In [ ]:
import os

with open('./data/data_train.json', 'r') as f:
    data_train = json.load(f)


with open('./data/data_val.json', 'r') as f:
    data_val = json.load(f)

images_number = len([name for name in os.listdir('./data/images/')])

print(f'Number of train configs: {len(data_train)}')
print(f'Number of validation configs: {len(data_val)}')
print(f'Total number of configs: {len(data_train) + len(data_val)}')
print(f'Number of images: {images_number}')

non_image_files = [name for name in os.listdir('./data/images/') if not name.endswith('.png') ]
print(f'Not image files {non_image_files}')

Number of train configs: 6630
Number of validation configs: 2211
Total number of configs: 8841
Number of images: 8841
Not image files []


## Initialize training and validation datasets

In [ ]:
train_dataset = KeypointsDataset('./data/images/', './data/data_train.json')
val_dataset = KeypointsDataset('./data/images/', './data/data_val.json')

In [ ]:
item, kps = train_dataset[0]
print(f'Item shape {item.shape}')
print(f'kps shape {kps.shape}')

Item shape torch.Size([3, 224, 224])
kps shape (28,)


## Training a model (Transfer learning)

We are going to adjust Resnet50 model with our data.
ResNet stands for Residual Network and is a specific type of convolutional neural network (CNN) introduced in the 2015.
ResNet-50 is a 50-layer convolutional neural network (48 convolutional layers, one MaxPool layer, and one average pool layer). Residual neural networks are a type of artificial neural network (ANN) that forms networks by stacking residual blocks.


Transfer learning means taking a pre-trained machine learning model and repurposing it for another related task for faster development. It helps achieve higher performance even if the model is trained on a smaller dataset.


### Fine-tunning
We get Resnet 50 pretrained model and utilize most of its layers that was trained on other dataset.
We are going to specify the last layer of Resnet 50. This layer is goint to be a Linear layer with input features equals
to last layers output features and **28 output features**(14 key points multiply by 2 for its coordinates (x, y coordinates)).

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torchmetrics

class ResNet50Lightning(pl.LightningModule):
  def __init__(self, pretrained=True, out_features=28, lr=3e-4, freeze=False) -> None:
      super(ResNet50Lightning, self).__init__()
      self.out_features = out_features
      self.lr = lr

      self.model = models.resnet50(pretrained=pretrained)
      if freeze:
          for param in self.model.parameters():
              param.requires_grad = False

      self.model.fc = torch.nn.Linear(self.model.fc.in_features, self.out_features)

      self.train_loss = torchmetrics.MeanSquaredError()
      self.val_loss = torchmetrics.MeanSquaredError()

      self.criterion = torch.nn.MSELoss()

  def configure_optimizers(self):
      optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2)
      return [optimizer], [scheduler]

  def forward(self, x):
      return self.model(x)

  def training_step(self, batch, batch_idx):
      x, y = batch
      preds = self.model(x)
      loss = self.criterion(preds, y)
      self.log('train_loss', loss.item(), on_epoch=True)
      return loss

  def validation_step(self, batch, batch_idx):
      x,y = batch
      preds = self.model(x)

      loss = self.criterion(preds, y)
      self.log('val_loss', loss.item(), on_epoch=True)

## Install and log into ClearML

In [ ]:
!pip install clearml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [ ]:
import clearml
clearml.browser_login()

<IPython.core.display.Javascript object>

ClearML automatic browser login failed, please login or create a new account
To get started with ClearML: setup your own `clearml-server`, or create a free account at https://app.clear.ml

Please login to https://app.clear.ml , then press [Enter] to connect dimodrangov@gmail.com


<IPython.core.display.Javascript object>


We cannot connect automatically (adblocker / incognito?) 😟 
Please go to https://app.clear.ml/settings/workspace-configuration 
Then press  + Create new credentials  
And copy/paste your Access Key here: 2VF4R7JNVMN4CP6L52PB
 Setting access key 
Now copy/paste your Secret Key here: spBrPpI1ILCxGB80kker6HBpvhL9MLXj4rx0Vuadj20vrNRZHw
 Setting secret key 


Hurrah! 🥳 🎊 🎉
🤖 ClearML connected successfully - let's build something! 🚀


## Train the model

First we are going to define a loss function, and since this is a **regression** task we will use **MSE (Mean squared error)**.

There we will specify Optimizer. We will use **Adam optimizer** with learning rate 0.001 and

In [ ]:
resnet=ResNet50Lightning(lr=0.001)
resnet.model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from clearml import Task
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger('tb_logs', name='resnet50-tennis-court-key-points')

Task.init(project_name="Tennis court keypoints Resnet", task_name="test experiment")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# It is strongly reccomended shuffle to be False for Validation and Test
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=2)
trainer = pl.Trainer(accelerator='gpu',
                     logger=logger,
                     max_epochs=10,
                     callbacks=[early_stopping],
                     log_every_n_steps=100)


trainer.fit(resnet, train_loader, val_loader)

ClearML Task: created new task id=58dd52cd533e4a94accc0dd32f871b74
2024-04-18 20:04:13,224 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/c5dda77c13314d768a68aef5d742823d/experiments/58dd52cd533e4a94accc0dd32f871b74/output/log


/usr/local/lib/python3.10/dist-packages/clearml/utilities/process/mp.py:622: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | model      | ResNet           | 23.6 M
1 | train_loss | MeanSquaredError | 0     
2 | val_loss   | MeanSquaredError | 0     
3 | criterion  | MSELoss          | 0     
-----------------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]